In [57]:
import mysql.connector
from mysql.connector.constants import ClientFlag

In [58]:
config = {
    'user': 'root',
    'password': 'Hh12H8y2ppBAq5Cj',
    'host': '34.65.175.109',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/server-ca.pem',
    'ssl_cert': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/client-cert.pem',
    'ssl_key': '/Users/xavier/Desktop/EPFL_Cours/Ici/DB/db-54/part2/ssl/client-key.pem',
    'database': 'data'
}

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

# Party
## Main Tables

In [49]:
cursor.execute("""CREATE TABLE Parties(id INTEGER,
                        case_id INTEGER NOT NULL,
                        party_number INTEGER NOT NULL,
                        finanicial_responsibility INTEGER,
                        party_age INTEGER,
                        party_sex INTEGER,
                        at_fault BIT NOT NULL,
                        cellphone_use INTEGER,
                        hazardous_materials CHAR(1),
                        movement_preceding_collision INTEGER,
                        other_associate_factor_1 INTEGER,
                        other_associate_factor_2 INTEGER,
                        party_drug_physical INTEGER,
                        party_safety_equipment_1 INTEGER,
                        party_safety_equipment_2 INTEGER,
                        party_sobriety INTEGER,
                        PRIMARY KEY(id),
                        FOREIGN KEY(finanicial_responsibility) REFERENCES FinancialResponsability(id),
                        FOREIGN KEY(party_sex) REFERENCES PersonSex(id),
                        FOREIGN KEY(cellphone_use) REFERENCES CellphoneUse(id),
                        FOREIGN KEY(movement_preceding_collision) REFERENCES MovementPrecedingCollision(id),
                        FOREIGN KEY(other_associate_factor_1) REFERENCES OtherAssociatedFactors(id),
                        FOREIGN KEY(other_associate_factor_2) REFERENCES OtherAssociatedFactors(id),
                        FOREIGN KEY(party_drug_physical) REFERENCES PartyDrugPhysical(id),
                        FOREIGN KEY(party_safety_equipment_1) REFERENCES PartySafetyEquipement(id),
                        FOREIGN KEY(party_safety_equipment_2) REFERENCES PartySafetyEquipement(id),
                        FOREIGN KEY(party_sobriety) REFERENCES PartySobriety(id),
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

In [50]:
cursor.execute("""CREATE TABLE Vehicles(party_id INTEGER NOT NULL,
                        school_bus_related BIT,
                        statewide_vehicle_type INTEGER,
                        vehicle_make INTEGER,
                        vehicle_year INTEGER,
                        FOREIGN KEY(statewide_vehicle_type) REFERENCES StatewideVehiculeType(id),
                        FOREIGN KEY(vehicle_make) REFERENCES VehiculeMake(id),
                        FOREIGN KEY(party_id) REFERENCES Parties(id))""")

## Satelite Enum Tables

In [11]:
cursor.execute("DROP TABLE OtherAssociatedFactors")
cursor.execute("CREATE TABLE OtherAssociatedFactors (id INTEGER,"
    "description CHAR(1) NOT NULL UNIQUE,"
    "CHECK (description BETWEEN 'A' AND 'Z'),"
    "PRIMARY KEY(id))")

In [12]:
cursor.execute("DROP TABLE CellphoneUse")
cursor.execute("""CREATE TABLE CellphoneUse(id INTEGER,
			   description CHAR(1) NOT NULL UNIQUE,
                CHECK (description BETWEEN 'B' AND 'D'),	
			   PRIMARY KEY(id))""")
# messy data: 1 -> B, 2 -> C, 3 -> D

In [14]:
cursor.execute("""CREATE TABLE FinancialResponsability(id INTEGER,
			   	      description CHAR(1) NOT NULL UNIQUE,
                                      CHECK (description IN ('N', 'Y', 'O', 'E', '-')),	
			              PRIMARY KEY(id))""")

# clean data 

In [16]:
cursor.execute("""CREATE TABLE MovementPrecedingCollision(id INTEGER,
					 description VARCHAR(100) NOT NULL UNIQUE,
					 PRIMARY KEY(id))""")
# varchar -> messy data, whatever.

In [17]:
cursor.execute("""CREATE TABLE PartyDrugPhysical(id INTEGER,
			       description CHAR(1) NOT NULL UNIQUE,
                            CHECK (description IN ('E', 'F', 'H', 'I')),
			       PRIMARY KEY(id))""")

# messy: letter G most prevalent: interpret as n/a ?

In [18]:
cursor.execute("""CREATE TABLE PartySafetyEquipement(id INTEGER,
				   description CHAR(1) NOT NULL UNIQUE,
                                   CHECK (description BETWEEN 'A' AND 'Y'),
				   PRIMARY KEY(id))""")
# clean as far as i can tell

The following table holds for `VictimSex` and `PartySex`

In [37]:
cursor.execute("""CREATE TABLE PersonSex(id INTEGER,
		       description VARCHAR(6) NOT NULL UNIQUE,
                       CHECK (description in ('male', 'female')),
		       PRIMARY KEY(id))""")
# clean af, for PartySex and VictimSex

In [20]:
cursor.execute("""CREATE TABLE PartySobriety(id INTEGER,
			    description CHAR(1) NOT NULL UNIQUE,
                            CHECK (description IN ('A', 'B', 'C', 'D', 'G', 'H')),
			    PRIMARY KEY(id))""")
# clean af

In [22]:
cursor.execute("""CREATE TABLE PartyType(id INTEGER,
			description VARCHAR(15) NOT NULL UNIQUE,
			PRIMARY KEY(id))""")

# clean

In [23]:
cursor.execute("""CREATE TABLE StatewideVehiculeType(id INTEGER,
				    description VARCHAR(30) NOT NULL UNIQUE,
				    PRIMARY KEY(id))""")
# clean as far as i can tell

In [24]:
cursor.execute("""CREATE TABLE VehiculeMake(id INTEGER,
			   description VARCHAR(100) NOT NULL,
			   PRIMARY KEY(id))""")

# clean

# Collision
## Main Tables

In [42]:
cursor.execute("""CREATE TABLE Collisions(case_id INTEGER, 
                        collision_date DATE NOT NULL,
                        collision_time TIME,
                        type_of_collision INTEGER,
                        collision_severity INTEGER NOT NULL,
                        hit_and_run INTEGER NOT NULL,
                        tow_away BIT,
                        FOREIGN KEY(type_of_collision) REFERENCES TypeOfCollision(id), 
                        FOREIGN KEY(collision_severity) REFERENCES CollisionSeverity(id), 
                        FOREIGN KEY(hit_and_run) REFERENCES HitAndRun(id),                      
                        PRIMARY KEY(case_id))""")

In [43]:
cursor.execute("""CREATE TABLE Pcfs(case_id INTEGER NOT NULL,
                    pcf_violation INTEGER,
                    pcf_violation_category INTEGER,
                    pcf_violation_subsection CHAR(1),
                    FOREIGN KEY(pcf_violation_category) REFERENCES PcfViolationCategory(id),
                    FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")
# pcf_violation_subscection has weird values but we cannot infer
# that they are wrong.

In [44]:
cursor.execute("""CREATE TABLE Locations(case_id INTEGER NOT NULL,
                        population INTEGER,
                        county_city_location INTEGER NOT NULL,
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

In [45]:
cursor.execute("""CREATE TABLE Factors(case_id INTEGER NOT NULL,
                        location_type INTEGER,
                        lighting INTEGER,
                        road_condition_1 INTEGER,
                        road_condition_2 INTEGER,
                        road_surface INTEGER,
                        weather_1 INTEGER,
                        weather_2 INTEGER,
                        FOREIGN KEY(location_type) REFERENCES LocationType(id),
                        FOREIGN KEY(lighting) REFERENCES Lighting(id),
                        FOREIGN KEY(road_condition_1) REFERENCES RoadCondition(id),
                        FOREIGN KEY(road_condition_2) REFERENCES RoadCondition(id),
                        FOREIGN KEY(road_surface) REFERENCES RoadSurface(id),
                        FOREIGN KEY(weather_1) REFERENCES Weather(id),
                        FOREIGN KEY(weather_2) REFERENCES Weather(id),
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

In [46]:
cursor.execute("""CREATE TABLE Cases(case_id INTEGER NOT NULL,
                    process_date DATE NOT NULL,
                    officer_id VARCHAR(8),
                    jurisdiction INTEGER,
                    FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")
# max officer_id length is 8. As some include letters, better make it VARCHAR.

## Satelite Enum Tables

In [61]:
cursor.execute("""CREATE TABLE CollisionSeverity(id INTEGER AUTO_INCREMENT,
			       description VARCHAR(25) NOT NULL UNIQUE,
			       PRIMARY KEY(id))""")
# clean

In [26]:
cursor.execute("""CREATE TABLE HitAndRun(id INTEGER AUTO_INCREMENT,
			description VARCHAR(20) NOT NULL UNIQUE,
			PRIMARY KEY(id))""")
# One dirty value: D -> NaN 

In [27]:
cursor.execute("""CREATE TABLE Lighting(id INTEGER AUTO_INCREMENT,
		       description VARCHAR(35) NOT NULL UNIQUE,
		       PRIMARY KEY(id))""")
# Clean

In [28]:
cursor.execute("""CREATE TABLE LocationType(id INTEGER AUTO_INCREMENT,
			   description VARCHAR(20) NOT NULL UNIQUE,
			   PRIMARY KEY(id))""")
# clean

In [29]:
cursor.execute("""CREATE TABLE PcfViolationCategory(id INTEGER AUTO_INCREMENT,
				   description VARCHAR(70) NOT NULL UNIQUE,
				   PRIMARY KEY(id))""")

# One dirty value: 21804 -> NaN

In [30]:
cursor.execute("""CREATE TABLE PrimaryCollisionFactor(id INTEGER,
				     description VARCHAR(25) NOT NULL UNIQUE,
				     PRIMARY KEY(id))""")
# clean

In [31]:
cursor.execute("""CREATE TABLE RoadCondition(id INTEGER,
			    description VARCHAR(20) NOT NULL UNIQUE,
			    PRIMARY KEY(id))""")
# clean

In [32]:
cursor.execute("""CREATE TABLE RoadSurface(id INTEGER,
			  description VARCHAR(15) NOT NULL UNIQUE,
			  PRIMARY KEY(id))""")
# one dirty value: H -> NaN

In [33]:
cursor.execute("""CREATE TABLE TypeOfCollision(id INTEGER,
			      description VARCHAR(15) NOT NULL UNIQUE,
			      PRIMARY KEY(id))""")
# clean

In [34]:
cursor.execute("""CREATE TABLE Weather(id INTEGER,
		      description VARCHAR(10) NOT NULL UNIQUE,
		      PRIMARY KEY(id))""")
# clean

# Victim
## Main Table

In [53]:
cursor.execute("""CREATE TABLE Victims(id INTEGER,
                        case_id INTEGER NOT NULL,
                        party_number INTEGER NOT NULL,
                        victim_age INTEGER,
                        victim_sex INTEGER,
                        victim_degree_of_injury INTEGER,
                        victim_ejected INTEGER,
                        victim_role INTEGER NOT NULL,
                        victim_safety_equipment_1 INTEGER,
                        victim_safety_equipment_2 INTEGER,
                        victim_seating_position INTEGER,
                        PRIMARY KEY(id),
                        FOREIGN KEY(victim_sex) REFERENCES PersonSex(id),
                        FOREIGN KEY(victim_safety_equipment_1) REFERENCES VictimSafetyEquipment(id),
                        FOREIGN KEY(victim_safety_equipment_2) REFERENCES VictimSafetyEquipment(id),
                        FOREIGN KEY(victim_degree_of_injury) REFERENCES VictimDegreeOfInjury(id),
                        FOREIGN KEY(case_id) REFERENCES Collisions(case_id))""")

## Satelite Table

In [52]:
cursor.execute("""CREATE TABLE VictimSafetyEquipment(id INTEGER,
				   description CHAR(1) NOT NULL UNIQUE,
                                   CHECK(description BETWEEN 'A' AND 'Z'),
				   PRIMARY KEY(id))""")

In [36]:
cursor.execute("""CREATE TABLE VictimDegreeOfInjury(id INTEGER,
				   description VARCHAR(30) NOT NULL UNIQUE,
				   PRIMARY KEY(id))""")
# one drity value: 7 -> NaN

# Data import
## Alterations to table definitions during data import

Only use `SET FOREIGN_KEY_CHECKS` during definition of tables, *not in production*

In [ ]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

cursor.execute("ALTER TABLE CollisionSeverity MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE HitAndRun MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE Lighting MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE LocationType MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PrimaryCollisionFactor MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE RoadCondition MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE RoadSurface MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE TypeOfCollision MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE Weather MODIFY id INTEGER AUTO_INCREMENT")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

In [63]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

cursor.execute("ALTER TABLE CellphoneUse MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE FinancialResponsability MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE MovementPrecedingCollision MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE OtherAssociatedFactors MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PartyDrugPhysical MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PartySafetyEquipement MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PersonSex MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PartySobriety MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE PartyType MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE StatewideVehiculeType MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE VehiculeMake MODIFY id INTEGER AUTO_INCREMENT")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

In [64]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")

cursor.execute("ALTER TABLE VictimSafetyEquipment MODIFY id INTEGER AUTO_INCREMENT")
cursor.execute("ALTER TABLE VictimDegreeOfInjury MODIFY id INTEGER AUTO_INCREMENT")

cursor.execute("SET FOREIGN_KEY_CHECKS = 1")

## Code to dump each csv row into the MySQL db

In [54]:
import pandas as pd

### Collision

In [104]:
def dump_pcfs(row):
    cursor.execute("""INSERT INTO PcfViolationCategory
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.pcf_violation_category))
    pcf_viol_categ_id = cursor.lastrowid

    cursor.execute("""INSERT INTO Pcfs
        VALUES (%s, %s, %s, %s)""", (row.case_id, row.pcf_violation, pcf_viol_categ_id, row.pcf_violation_subsection))

In [105]:
def dump_locations(row):
    cursor.execute("""INSERT INTO Pcfs
        VALUES (%s, %s, %s)""", (row.case_id, row.population, row.county_city_location))

In [106]:
def dump_factors(row):
    cursor.execute("""INSERT INTO LocationType
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.location_type,))
    location_type_id = cursor.lastrowid

    cursor.execute("""INSERT INTO Lighting
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.lighting,))
    lighting_id = cursor.lastrowid

    cursor.execute("""INSERT INTO RoadCondition
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.road_condition_1,))
    road_cond1_id = cursor.lastrowid

    cursor.execute("""INSERT INTO RoadCondition
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.road_condition_2,))
    road_cond2_id = cursor.lastrowid

    cursor.execute("""INSERT INTO RoadSurface
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.road_surface,))
    road_surface_id = cursor.lastrowid

    cursor.execute("""INSERT INTO Weather
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.weather_1,))
    weather1_id = cursor.lastrowid

    cursor.execute("""INSERT INTO Weather
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.weather_2,))
    weather2_id = cursor.lastrowid

    cursor.execute("""INSERT INTO Factors
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""",(row.case_id, location_type_id, lighting_id, road_cond1_id, road_cond2_id, road_surface_id, weather1_id, weather2_id))

In [107]:
def dump_cases(row):
    cursor.execute("""INSERT INTO Cases
        VALUES (%s, %s, %s, %s)""",(row.case_id, row.process_date, row.officer_id, row.jurisdiction))

In [117]:
def dump_collision_row(row):
    cursor.execute("""INSERT INTO TypeOfCollision 
        (description)
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.type_of_collision,))
    col_type_id = cursor.lastrowid

    cursor.execute("""INSERT INTO CollisionSeverity
        (description)
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.collision_severity,))
    col_severity_id = cursor.lastrowid

    cursor.execute("""INSERT INTO HitAndRun
        (description)
        VALUES (%s)
        ON DUPLICATE KEY UPDATE id=id""", (row.hit_and_run,))
    hit_n_run_id = cursor.lastrowid

    print(col_type_id)
    print(col_severity_id)
    print(hit_n_run_id)

    cursor.execute("""INSERT INTO Collisions
        VALUES (%s, %s, %s, %s, %s, %s, %s)""", (row.case_id, row.collision_date, row.collision_time, col_type_id, col_severity_id, hit_n_run_id, row.tow_away))

    dump_pcfs(row)
    dump_locations(row)
    dump_factors(row)
    dump_cases(row)

In [109]:
def import_collisions(df):
    for row in df.itertuples(index=True, name="Pandas"):
        dump_collision_row(row)

In [55]:
collisions = pd.read_csv('../collisions2018.csv')

/Users/xavier/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [120]:
cursor.execute("TRUNCATE TABLE Collisions")
cursor.execute("TRUNCATE TABLE TypeOfCollision")
cursor.execute("TRUNCATE TABLE CollisionSeverity")
cursor.execute("TRUNCATE TABLE HitAndRun")

ProgrammingError: 1701 (42000): Cannot truncate a table referenced in a foreign key constraint (`data`.`Pcfs`, CONSTRAINT `Pcfs_ibfk_2`)

In [118]:
import_collisions(collisions)

0
0
0


IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`data`.`Collisions`, CONSTRAINT `Collisions_ibfk_1` FOREIGN KEY (`type_of_collision`) REFERENCES `TypeOfCollision` (`id`))

In [ ]:
parties = pd.read_csv('../parties2018.csv')

In [ ]:
def import_parties():
    for row in parties.itertuples(index=True, name="Pandas"):

In [ ]:
victims = pd.read_csv('../victims2018.csv')

In [ ]:
def import_victims(col):
    for row in victims.itertuples(index=True, name="Pandas"):